In [29]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [30]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [31]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [32]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [33]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [34]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [35]:
df['loc']

1     (9001 Collins Ave s 511, Surfside, FL 33154, U...
2     (19575 Collins Ave PH 43, Sunny Isles Beach, F...
3     (8701 Collins Ave #1501, Miami Beach, FL 33154...
4     (5959 Collins Ave APT 1407, Miami Beach, FL 33...
5     (20165 NE 39th Pl, Aventura, FL 33180, USA, (2...
6     (102 24th St, Miami Beach, FL 33139, USA, (25....
7     (10 Edgewater Dr #3a, Miami, FL 33133, USA, (2...
8     (18501 Collins Ave #3903, Sunny Isles Beach, F...
9     (1660 W Glencoe St #301, Miami, FL 33133, USA,...
10    (10201 Collins Ave #2107w, Bal Harbour, FL 331...
Name: loc, dtype: object

In [36]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [37]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Surf Club Condo\r9001 Collins Ave S-511\rSurfside,9/29/2021,2/15/2022,83,"$23,000,000.00",4668,"$4,927.16",Roberta Ingletto,Michael Duchon,Surf Club Condo 9001 Collins Ave S-511 Surfside,Surf Club Condo,9001 Collins Ave S-511 Surfside,"(9001 Collins Ave s 511, Surfside, FL 33154, U...","(25.8768949, -80.12146349999999, 0.0)",25.876895,-80.121463,0.0
2,Regalia Condo\r19575 Collins Ave PH-43\rSunny ...,6/10/2021,2/16/2022,196,"$20,524,000.00",9193,"$2,232.57",Jeffrey Miller,Mark Pordes,Regalia Condo 19575 Collins Ave PH-43 Sunny Is...,Regalia Condo,19575 Collins Ave PH-43 Sunny Isles Beach,"(19575 Collins Ave PH 43, Sunny Isles Beach, F...","(25.956929, -80.1193868, 0.0)",25.956929,-80.119387,0.0
3,87 Park\r8701 Collins Ave 1501\rMiami Beach,6/8/2021,2/14/2022,210,"$9,250,000.00",3140,"$2,945.86",Lourdes Alatriste,Ivan Chorney,87 Park 8701 Collins Ave 1501 Miami Beach,,87 Park 8701 Collins Ave 1501 Miami Beach,"(8701 Collins Ave #1501, Miami Beach, FL 33154...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0
4,The Bath Club\r5959 Collins Ave 1407\rMiami Beach,10/8/2019,2/14/2022,789,"$5,894,250.00",4863,"$1,212.06",Jill Eber,Amber Taylor,The Bath Club 5959 Collins Ave 1407 Miami Beach,The Bath Club,5959 Collins Ave 1407 Miami Beach,"(5959 Collins Ave APT 1407, Miami Beach, FL 33...","(25.8424476, -80.1203048, 0.0)",25.842448,-80.120305,0.0
5,Porto Vita\r20165 NE 39th Pl TS-1\rAventura,8/28/2019,2/14/2022,807,"$5,500,000.00",9600,$572.92,Debbie Lazoff PA,Sylvia Fragos,Porto Vita 20165 NE 39th Pl TS-1 Aventura,Porto Vita,20165 NE 39th Pl TS-1 Aventura,"(20165 NE 39th Pl, Aventura, FL 33180, USA, (2...","(25.9624882, -80.1246262, 0.0)",25.962488,-80.124626,0.0
6,1 Hotel & Homes\r102 24th St 1419\rMiami Beach,7/21/2021,2/14/2022,146,"$5,100,000.00",2378,"$2,403.39",Tracy Galya,Tracy Galya,1 Hotel & Homes 102 24th St 1419 Miami Beach,,1 Hotel & Homes 102 24th St 1419 Miami Beach,"(102 24th St, Miami Beach, FL 33139, USA, (25....","(25.7991784, -80.1272173, 0.0)",25.799178,-80.127217,0.0
7,The Gables Club Condo\r10 Edgewater Dr 4A/3A\r...,3/5/2020,2/15/2022,691,"$4,850,000.00",7660,$633.16,Audrey Ross,Magnus Jennemyr,The Gables Club Condo 10 Edgewater Dr 4A/3A Co...,The Gables Club Condo,10 Edgewater Dr 4A/3A Coral Gables,"(10 Edgewater Dr #3a, Miami, FL 33133, USA, (2...","(25.706719, -80.25356099999999, 0.0)",25.706719,-80.253561,0.0
8,Turnberry Ocean Club\r18501 Collins Ave 3903\r...,11/2/2021,2/15/2022,71,"$4,800,000.00",3760,"$1,543.41",Diane Cohen,Yackeline Leiderman,Turnberry Ocean Club 18501 Collins Ave 3903 Su...,Turnberry Ocean Club,18501 Collins Ave 3903 Sunny Isles Beach,"(18501 Collins Ave #3903, Sunny Isles Beach, F...","(25.9477873, -80.12022019999999, 0.0)",25.947787,-80.120220,0.0
9,Beacon Harbour\r1660 W Glencoe St 301\rMiami,12/30/2021,2/16/2022,37,"$4,150,000.00",3812,"$1,088.67",Arai Penate,Megan Probst,Beacon Harbour 1660 W Glencoe St 301 Miami,Beacon Harbour,1660 W Glencoe St 301 Miami,"(1660 W Glencoe St #301, Miami, FL 33133, USA,...","(25.7397943, -80.2183617, 0.0)",25.739794,-80.218362,0.0
10,Oceana Bal Harbour\r10201 Collins Ave 2107W\rB...,4/8/2019,2/14/2022,989,"$4,000,000.00",2226,"$1,796.95",Olga Mirer,Oren Alexander,Oceana Bal Harbour 10201 Collins Ave 2107W Bal...,Oceana Bal Harbour,10201 Collins Ave 2107W Bal Harbour,"(10201 Collins Ave #2107w, Bal Harbour, FL 331...","(25.8951677, -80.12315, 0.0)",25.895168,-80.123150,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [41]:
df.at[6,'building_name']=('1 Hotel & Homes')


In [38]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [42]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February 13th - February 19th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [40]:
m.save('index.html')